<a href="https://colab.research.google.com/github/tzf101/BDA-Bangla-Text-Data-Augmentation/blob/main/utils_notebook/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

In [ ]:
!pip install bnlp

In [ ]:
!pip install bnlp-toolkit

#Imports

In [ ]:
import random
from random import shuffle
import re

In [ ]:
from bnlp import BengaliCorpus as corpus

In [ ]:
# import nltk
# from nltk.corpus import wordnet
# nltk.download('wordnet')

In [ ]:
# def get_bangla_synonyms(word):
#     synonyms = []
#     for synset in wordnet.synsets(word, lang='bwn'):
#         for lemma in synset.lemmas('ben'):
#             synonyms.append(lemma.name())
#     return list(set(synonyms))  # Remove duplicates

# # Example usage
# word_to_query = "খুশি"
# synonyms = get_bangla_synonyms(word_to_query)

# print(f"Synonyms for '{word_to_query}': {synonyms}")

# Preprocessing

In [ ]:
punc = corpus.punctuations + ("‘") + ("’")

In [ ]:
STOPWORDS = set(corpus.stopwords)
PUNCTUATIONS = set(punc)

In [ ]:
class CleanText:
    def __init__(self):
        self.PUNCTUATIONS = set(punc)
        self.STOPWORDS = set(corpus.stopwords)

    def remove_digits(self, text):
        return re.sub(r'[০-৯]+\d+', '', text).strip()

    def remove_punctuations(self, text, replace_with=" "):
        for punc in self.PUNCTUATIONS:
            text = text.replace(punc, replace_with)
        return ' '.join(text.split())

    def remove_stopwords(self, text):
        words = text.split()
        new_text = [word for word in words if word.lower() not in self.STOPWORDS]
        return ' '.join(new_text)

In [ ]:
ct = CleanText()

In [ ]:
text = "সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে।"
# text_np = ct.remove_digits(text)
text_np = ct.remove_punctuations(text)
print(text_np)

সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে


In [ ]:
words = text_np.split()
len(words)

13

# Random Deletion

In [ ]:
def random_deletion(text, p=0.3):
    words = text.split()
    #obviously, if there's only one word, don't delete it
    if len(words) == 1:
      return words

    #randomly delete words with probability p
    new_words = []
    for word in words:
      r = random.uniform(0, 1)
      if r > p:
        new_words.append(word)

    #if you end up deleting all words, just return a random word
    if len(new_words) == 0:
      rand_int = random.randint(0, len(words)-1)
      return [words[rand_int]]

    return ' '.join(new_words) + "(rd)"

In [ ]:
rd_text = random_deletion(text_np, 0.3)
print(rd_text)

সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে যে এই করেছে(rd)


# Random Swap

In [ ]:
def random_swap(text, n):
    words = text.split()
    new_words = words.copy()
    for _ in range(n):
      new_words = swap_word(new_words)
    return ' '.join(new_words) + "(rs)"

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random.randint(0, len(new_words)-1)
    counter = 0
    while random_idx_2 == random_idx_1:
      random_idx_2 = random.randint(0, len(new_words)-1)
      counter += 1
      if counter > 3:
        return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

In [ ]:
rs_text = random_swap(text_np, 1)
print(rs_text)

সঠিক কাজ করতে হবে বিচারের আওতায় আনতে হবে যে এই তদন্ত টা করেছে(rs)


# Synonym Replacement


In [ ]:
#testing
from bnlp import BengaliWord2Vec

bwv = BengaliWord2Vec()

In [ ]:
word = 'গ্রাম'
syn = set()
similar_words = bwv.get_most_similar_words(word, topn=10)
for word in similar_words:
  syn.add(word[0])
print(syn)

{'গ্রামের', 'গ্রামে', 'মহল্লা', 'পুরসভা', 'তহসিল', 'তালুক', 'সংগ্রামপুর', 'মৌজা', 'গ্রামাঞ্চল', 'মৌজার'}


In [ ]:
def synonym_replacement(text, n):
    words = text.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in STOPWORDS]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
      synonyms = get_synonyms(random_word)
      if len(synonyms) >= 1:
        synonym = random.choice(list(synonyms))
        new_words = [synonym if word == random_word else word for word in new_words]
        #print("replaced", random_word, "with", synonym)
        num_replaced += 1
      if num_replaced >= n: #only replace up to n words
        break

    #this is stupid but we need it, trust me
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')

    return ' '.join(new_words) + "(sr)"

def get_synonyms(word):
    synonyms = set()
    similar_words = bwv.get_most_similar_words(word, topn=10)
    for word in similar_words:
      synonyms.add(word[0])
    if word in synonyms:
      synonyms.remove(word)
    return list(synonyms)

In [ ]:
print(synonym_replacement(text_np, 2))

সঠিক তদন্তের করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ ট্রুপ- করেছে(sr)


# Random Insertion

In [ ]:
def random_insertion(text, n):
    words = text.split()
    new_words = words.copy()
    for _ in range(n):
      add_word(new_words)
    return ' '.join(new_words) + "(ri)"

def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
      random_word = new_words[random.randint(0, len(new_words)-1)]
      synonyms = get_synonyms(random_word)
      counter += 1
      if counter >= 10:
        return
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

In [ ]:
print(random_insertion(text_np, 2))

সঠিক তদন্ত করতে হবে বিচারের আওতায় কিভাবে আনতে হবে যে এই কাজ টা শুনানি করেছে(ri)


# Main Data Augmentation Function

In [ ]:
def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):

    ct = CleanText()
    text = ct.remove_punctuations(sentence)

    words = text.split()
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug/4)+1

    #sr
    if (alpha_sr > 0):
      n_sr = max(1, int(alpha_sr*num_words))
      for _ in range(num_new_per_technique):
        augmented_sentences.append(synonym_replacement(text, n_sr))

    #ri
    if (alpha_ri > 0):
      n_ri = max(1, int(alpha_ri*num_words))
      for _ in range(num_new_per_technique):
        augmented_sentences.append(random_insertion(text, n_ri))

    #rs
    if (alpha_rs > 0):
      n_rs = max(1, int(alpha_rs*num_words))
      for _ in range(num_new_per_technique):
        augmented_sentences.append(random_swap(text, n_rs))

    # print(augmented_sentences)

    #rd
    if (p_rd > 0):
      for _ in range(num_new_per_technique):
        augmented_sentences.append(random_deletion(text, p_rd))

    # print(augmented_sentences)

    # augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
    shuffle(augmented_sentences)


    #trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
      augmented_sentences = augmented_sentences[:num_aug]
    else:
      keep_prob = num_aug / len(augmented_sentences)
      augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
    #append the original sentence
    augmented_sentences.append(sentence)

    return augmented_sentences

# Testing

In [ ]:
augs = eda(text)

In [ ]:
len(augs)

10

In [ ]:
for aug in augs:
  print(aug)

সঠিক তদন্ত বিচারের হবে করতে আওতায় আনতে হবে যে এই কাজ টা করেছে(rs)
সঠিক তদন্ত করতে গ্রেফতারের হবে বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে(ri)
সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ টা পদ- করেছে(ri)
সঠিক করতে হবে বিচারের আওতায় আনতে হবে যে এই টা করেছে(rd)
টা তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ সঠিক করেছে(rs)
সঠিক তদন্ত বিচারের আওতায় আনতে হবে যে এই কাজ করেছে(rd)
সঠিক তদন্ত করতে হবে শুনানি বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে(ri)
সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ(rd)
সঠিক তদন্ত এই হবে বিচারের আওতায় আনতে হবে যে করতে কাজ টা করেছে(rs)
সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে।
